In [46]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import warnings
warnings.simplefilter(action= 'ignore', category='SettingWithCopyWarning')


from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score, accuracy_score

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier
from itertools import product



s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key1= 'churn-bigml-80.csv'
file_key2= 'churn-bigml-20.csv'


bucket_object1= bucket.Object(file_key1)
file_object1= bucket_object1.get()
file_content_stream1 = file_object1.get('Body')

bucket_object2= bucket.Object(file_key2)
file_object2 = bucket_object2.get()
file_content_stream2 = file_object2.get('Body')

#reading the datefile
telecom_train = pd.read_csv(file_content_stream1)
telecom_test= pd.read_csv(file_content_stream2)

In [30]:
telecom_train= pd.DataFrame(telecom_train)
telecom_test= pd.DataFrame(telecom_test)

In [31]:
#Creating churn_numb true/false to 1/0
telecom_train= telecom_train.assign(churn_numb= telecom_train['Churn'].astype(int))
telecom_test= telecom_test.assign(churn_numb= telecom_test['Churn'].astype(int))

#Changing International_plan yes/no to 1/0
telecom_train['International_plan'].replace(['Yes', 'No'], [1,0], inplace= True)
telecom_test['International_plan'].replace(['Yes', 'No'], [1,0], inplace= True)

#Changing Voice_mail_plan yes/no to 1/0
telecom_train['Voice_mail_plan'].replace(['Yes', 'No'], [1,0], inplace= True)
telecom_test['Voice_mail_plan'].replace(['Yes', 'No'], [1,0], inplace= True)

#Creating variable Total_charge
telecom_train= telecom_train.assign(total_charge= telecom_train['Total_day_charge'] + telecom_train['Total_eve_charge'] + telecom_train['Total_night_charge']+ telecom_train['Total_intl_charge'])
telecom_test= telecom_test.assign(total_charge= telecom_test['Total_day_charge'] + telecom_test['Total_eve_charge'] + telecom_test['Total_night_charge']+ telecom_test['Total_intl_charge'])

In [32]:
#Setting x and y variables
x = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
y = telecom_train['churn_numb']

In [ ]:
rf_results= list()
ada_results= list()
gb_results= list()

for i in range(0,1000):
    #Splitting the Data
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, stratify= y)
    
    #Buidling the model
    rf_md= RandomForestClassifier(n_estimators= 500, max_depth= 3).fit(x_train, y_train)
    #Extracting the feature importances
    rf_results.append(rf_md.feature_importances_)
    
    #Building the model
    ada_md= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Extracting the feature importances
    ada_results.append(ada_md.feature_importances_)
    
    
    #Building the model
    gb_md= GradientBoostingClassifier(max_depth= 3, n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Extracting the feature importances
    gb_results.append(gb_md.feature_importances_)
    

In [26]:
a = pd.DataFrame(rf_results)
a.columns= ['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']
a.apply(np.mean, axis = 0)

NameError: name 'rf_results' is not defined

In [ ]:
b = pd.DataFrame(ada_results)
b.columns= ['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']
b.apply(np.mean, axis = 0)

In [ ]:
c = pd.DataFrame(gb_results)
c.columns= ['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']
c.apply(np.mean, axis = 0)

In [ ]:
#Top 4 average importances: international_plan, voice_mail_plan, total_charge, customer_service_calls
    #I dropped Account_length since it was insignificant in 2/3 models

In [33]:
x= x.drop(columns= 'Account_length')

In [34]:
def expand_grid(dictionary1):
    return pd.DataFrame([row for row in product(*dictionary1.values())], 
                        columns = dictionary1.keys())
dictionary1 = {'n_tree': [100, 500, 1000, 1500, 2000], 
                'depth': [3, 5, 7]}
parameters1 = expand_grid(dictionary1)
parameters1['recall']= np.nan
parameters1['accuracy']= np.nan

In [35]:
parameters1

,n_tree,depth,recall,accuracy
0,100,3,NaN,NaN
1,100,5,NaN,NaN
2,100,7,NaN,NaN
3,500,3,NaN,NaN
4,500,5,NaN,NaN
5,500,7,NaN,NaN
6,1000,3,NaN,NaN
7,1000,5,NaN,NaN
8,1000,7,NaN,NaN
9,1500,3,NaN,NaN


In [49]:
def expand_grid(dictionary2):
    return pd.DataFrame([row for row in product(*dictionary2.values())], 
                        columns = dictionary2.keys())
dictionary2 = {'n_tree': [100, 500, 1000, 1500, 2000], 
                'depth': [3, 5, 7], 
                'learning_rate': [0.1, 0.01, 0.001]}
parameters2 = expand_grid(dictionary2)
parameters2['recall']= np.nan
parameters2['accuracy']= np.nan

In [50]:
parameters2

,n_tree,depth,learning_rate,recall,accuracy
0,100,3,0.100,NaN,NaN
1,100,3,0.010,NaN,NaN
2,100,3,0.001,NaN,NaN
3,100,5,0.100,NaN,NaN
4,100,5,0.010,NaN,NaN
5,100,5,0.001,NaN,NaN
6,100,7,0.100,NaN,NaN
7,100,7,0.010,NaN,NaN
8,100,7,0.001,NaN,NaN
9,500,3,0.100,NaN,NaN


In [ ]:
def expand_grid(dictionary3):
    return pd.DataFrame([row for row in product(*dictionary3.values())],
                        columns = dictionary3.keys())
dictionary3 = {'n_tree': [100, 500, 1000, 1500, 2000],
                'depth': [3, 5, 7], 
                'learning_rate': [0.1, 0.01, 0.001]}
parameters3 = expand_grid(dictionary3)
parameters3['recall']= np.nan
parameters3['accuracy']= np.nan

In [ ]:
parameters3

In [53]:
for i in range (0,10):

    #Splitting the Data
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, stratify= y)
    

    ##### Random Forest
    for k in range(len(parameters1)):
        #Buidling the model
        rf_md1= RandomForestClassifier(n_estimators= parameters1['n_tree'][k], max_depth=parameters1['depth'][k]).fit(x_train, y_train)
        #Predicting on the model
        rf_pred1= rf_md1.predict_proba(x_test)[:,1]
        #changing liklihoods to labels 
        rf_labels1= np.where(rf_pred1 < .1, 0, 1)
        parameters1['recall'][k]= recall_score(y_test, rf_labels1)
        parameters1['accuracy'][k]= accuracy_score(y_test, rf_labels1)
        
    #### Ada Boost    
    for m in range(len(paramters2)):
        #Building the model
        ada_md1= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= parameters2['n_tree'][m]), n_estimators= parameters2['depth'][m], learning_rate= parameters2['learning_rate'][m]).fit(x_train, y_train)
        #Predicting on the model
        ada_pred1= ada_md1.predict_proba(x_test)[:,1]
        #changing liklihoods to labels 
        ada_labels1= np.where(ada_pred1 < .1, 0, 1)
        paramters2['recall'][m]= recall_score(y_test, ada_labels1)
        paramters2['accuracy'][m]= accuracy_score(y_test, ada_labels1)

    ### Gradient Boost
    for n in range(len(paramters3)):
        #Building the model
        gb_md= GradientBoostingClassifier(max_depth= parameters3['depth'][n], n_estimators= parameters3['n_tree'][n], learning_rate= parameters3['learning_rate'][n]).fit(x_train, y_train)
        #Predicting on the model
        gb_pred= gb_md.predict_proba(x_test)[:,1]
        #changing liklihoods to labels 
        gb_labels= np.where(gb_pred < .1, 0, 1)
        paramters3['recall'][n]= recall_score(y_test, gb_labels)
        paramters3['accuracy'][n] = accuracy_score(y_test, gb_labels)

TypeError: issubclass() arg 2 must be a class or tuple of classes

In [5]:
rf_100_3_recall= list()
rf_100_3_accuracy= list()

rf_100_5_recall= list()
rf_100_5_accuracy= list()

rf_100_7_recall= list()
rf_100_7_accuracy= list()

rf_500_3_recall= list()
rf_500_3_accuracy= list()

rf_500_5_recall= list()
rf_500_5_accuracy= list()

rf_500_7_recall= list()
rf_500_7_accuracy= list()

rf_1000_3_accuracy= list()
rf_1000_3_recall= list()

rf_1000_5_recall= list()
rf_1000_5_accuracy= list()

rf_1000_7_recall= list()
rf_1000_7_accuracy= list()

rf_1500_3_recall= list()
rf_1500_3_accuracy= list()

rf_1500_5_recall= list()
rf_1500_5_accuracy= list()

rf_1500_7_recall= list()
rf_1500_7_accuracy= list()

rf_2000_3_recall= list()
rf_2000_3_accuracy= list()

rf_2000_5_recall= list()
rf_2000_5_accuracy= list()

rf_2000_7_recall= list()
rf_2000_7_accuracy= list()



for i in range (0,100):

    #Splitting the Data
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, stratify= y)
    
##### Building random forest models  

    ##### 100
    
    #Buidling the model
    rf_md1= RandomForestClassifier(n_estimators= 100, max_depth= 3).fit(x_train, y_train)
    #Predicting on the model
    rf_pred1= rf_md1.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels1= np.where(rf_pred1 < .1, 0, 1)
    rf_100_3_recall.append(recall_score(y_test, rf_labels1))
    rf_100_3_accuracy.append(accuracy_score(y_test, rf_labels1))
    
    #Buidling the model
    rf_md2= RandomForestClassifier(n_estimators= 100, max_depth= 5).fit(x_train, y_train)
    #Predicting on the model
    rf_pred2= rf_md2.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels2= np.where(rf_pred2 < .1, 0, 1)
    rf_100_5_recall.append(recall_score(y_test, rf_labels2))
    rf_100_5_accuracy.append(accuracy_score(y_test, rf_labels2))
    
    #Buidling the model
    rf_md3= RandomForestClassifier(n_estimators= 100, max_depth= 7).fit(x_train, y_train)
    #Predicting on the model
    rf_pred3= rf_md3.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels3= np.where(rf_pred3 < .1, 0, 1)
    rf_100_7_recall.append(recall_score(y_test, rf_labels3))
    rf_100_7_accuracy.append(accuracy_score(y_test, rf_labels3))
    
    
    ###### 500
    
     #Buidling the model
    rf_md4= RandomForestClassifier(n_estimators= 500, max_depth= 3).fit(x_train, y_train)
    #Predicting on the model
    rf_pred4= rf_md4.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels4= np.where(rf_pred4 < .1, 0, 1)
    rf_500_3_recall.append(recall_score(y_test, rf_labels4))
    rf_500_3_accuracy.append(accuracy_score(y_test, rf_labels4))
    
    #Buidling the model
    rf_md5= RandomForestClassifier(n_estimators= 500, max_depth= 5).fit(x_train, y_train)
    #Predicting on the model
    rf_pred5= rf_md5.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels5= np.where(rf_pred5 < .1, 0, 1)
    rf_500_5_recall.append(recall_score(y_test, rf_labels5))
    rf_500_5_accuracy.append(accuracy_score(y_test, rf_labels5))
    
    #Buidling the model
    rf_md6= RandomForestClassifier(n_estimators= 500, max_depth= 7).fit(x_train, y_train)
    #Predicting on the model
    rf_pred6= rf_md6.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels6= np.where(rf_pred6 < .1, 0, 1)
    rf_500_7_recall.append(recall_score(y_test, rf_labels6))
    rf_500_7_accuracy.append(accuracy_score(y_test, rf_labels6))
    
    
    ###### 1000
    
     #Buidling the model
    rf_md7= RandomForestClassifier(n_estimators= 1000, max_depth= 3).fit(x_train, y_train)
    #Predicting on the model
    rf_pred7= rf_md7.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels7= np.where(rf_pred7 < .1, 0, 1)
    rf_1000_3_recall.append(recall_score(y_test, rf_labels7))
    rf_1000_3_accuracy.append(accuracy_score(y_test, rf_labels7))
    
    #Buidling the model
    rf_md8= RandomForestClassifier(n_estimators= 1000, max_depth= 5).fit(x_train, y_train)
    #Predicting on the model
    rf_pred8= rf_md8.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels8= np.where(rf_pred8 < .1, 0, 1)
    rf_1000_5_recall.append(recall_score(y_test, rf_labels8))
    rf_1000_5_accuracy.append(accuracy_score(y_test, rf_labels8))
    
    #Buidling the model
    rf_md9= RandomForestClassifier(n_estimators= 1000, max_depth= 7).fit(x_train, y_train)
    #Predicting on the model
    rf_pred9= rf_md9.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels9= np.where(rf_pred9 < .1, 0, 1)
    rf_1000_7_recall.append(recall_score(y_test, rf_labels9))
    rf_1000_7_accuracy.append(accuracy_score(y_test, rf_labels9))
    
    
    
    ####### 1500
    
    
     #Buidling the model
    rf_md10= RandomForestClassifier(n_estimators= 1500, max_depth= 3).fit(x_train, y_train)
    #Predicting on the model
    rf_pred10= rf_md10.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels10= np.where(rf_pred10 < .1, 0, 1)
    rf_1500_3_recall.append(recall_score(y_test, rf_labels10))
    rf_1500_3_accuracy.append(accuracy_score(y_test, rf_labels10))
    
    #Buidling the model
    rf_md11= RandomForestClassifier(n_estimators= 1500, max_depth= 5).fit(x_train, y_train)
    #Predicting on the model
    rf_pred11= rf_md11.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels11= np.where(rf_pred11 < .1, 0, 1)
    rf_1500_5_recall.append(recall_score(y_test, rf_labels11))
    rf_1500_5_accuracy.append(accuracy_score(y_test, rf_labels11))
    
    #Buidling the model
    rf_md12= RandomForestClassifier(n_estimators= 1500, max_depth= 7).fit(x_train, y_train)
    #Predicting on the model
    rf_pred12= rf_md12.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels12= np.where(rf_pred12 < .1, 0, 1)
    rf_1500_7_recall.append(recall_score(y_test, rf_labels12))
    rf_1500_7_accuracy.append(accuracy_score(y_test, rf_labels12))
    
    
    ##### 2000
    
     #Buidling the model
    rf_md13= RandomForestClassifier(n_estimators= 2000, max_depth= 3).fit(x_train, y_train)
    #Predicting on the model
    rf_pred13= rf_md13.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels13= np.where(rf_pred13 < .1, 0, 1)
    rf_2000_3_recall.append(recall_score(y_test, rf_labels13))
    rf_2000_3_accuracy.append(accuracy_score(y_test, rf_labels13))
    
    #Buidling the model
    rf_md14= RandomForestClassifier(n_estimators= 2000, max_depth= 5).fit(x_train, y_train)
    #Predicting on the model
    rf_pred14= rf_md14.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels14= np.where(rf_pred14 < .1, 0, 1)
    rf_2000_5_recall.append(recall_score(y_test, rf_labels14))
    rf_2000_5_accuracy.append(accuracy_score(y_test, rf_labels14))
    
    #Buidling the model
    rf_md15= RandomForestClassifier(n_estimators= 2000, max_depth= 7).fit(x_train, y_train)
    #Predicting on the model
    rf_pred15= rf_md15.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels15= np.where(rf_pred15 < .1, 0, 1)
    rf_2000_7_recall.append(recall_score(y_test, rf_labels15))
    rf_2000_7_accuracy.append(accuracy_score(y_test, rf_labels15))

In [6]:
print('Average Recall for Random Forest model 100,3:', np.mean(rf_100_3_recall))
print('Average Accuracy for Random Forest model 100,3:', np.mean(rf_100_3_accuracy))
print('Average Recall for Random Forest model 100,35:', np.mean(rf_100_5_recall))
print('Average Accuracy for Random Forest model 100,5:', np.mean(rf_100_5_accuracy))
print('Average Recall for Random Forest model 100,7:', np.mean(rf_100_7_recall))
print('Average Accuracy for Random Forest model 100,7:', np.mean(rf_100_7_accuracy))

Average Recall for Random Forest model 100,3: 0.864871794871795
Average Accuracy for Random Forest model 100,3: 0.871460674157303
Average Recall for Random Forest model 100,35: 0.8650000000000001
Average Accuracy for Random Forest model 100,5: 0.8752434456928836
Average Recall for Random Forest model 100,7: 0.863846153846154
Average Accuracy for Random Forest model 100,7: 0.9015730337078653


In [7]:
print('Average Recall for Random Forest model 500,3:', np.mean(rf_500_3_recall))
print('Average Accuracy for Random Forest model 500,3:', np.mean(rf_500_3_accuracy))
print('Average Recall for Random Forest model 500,35:', np.mean(rf_500_5_recall))
print('Average Accuracy for Random Forest model 500,5:', np.mean(rf_500_5_accuracy))
print('Average Recall for Random Forest model 500,7:', np.mean(rf_500_7_recall))
print('Average Accuracy for Random Forest model 500,7:', np.mean(rf_500_7_accuracy))

Average Recall for Random Forest model 500,3: 0.8647435897435899
Average Accuracy for Random Forest model 500,3: 0.871516853932584
Average Recall for Random Forest model 500,35: 0.8647435897435899
Average Accuracy for Random Forest model 500,5: 0.8732771535580521
Average Recall for Random Forest model 500,7: 0.8642307692307694
Average Accuracy for Random Forest model 500,7: 0.9033333333333335


In [8]:
print('Average Recall for Random Forest model 1000,3:', np.mean(rf_1000_3_recall))
print('Average Accuracy for Random Forest model 1000,3:', np.mean(rf_1000_3_accuracy))
print('Average Recall for Random Forest model 1000,35:', np.mean(rf_1000_5_recall))
print('Average Accuracy for Random Forest model 1000,5:', np.mean(rf_1000_5_accuracy))
print('Average Recall for Random Forest model 1000,7:', np.mean(rf_1000_7_recall))
print('Average Accuracy for Random Forest model 1000,7:', np.mean(rf_1000_7_accuracy))

Average Recall for Random Forest model 1000,3: 0.8650000000000001
Average Accuracy for Random Forest model 1000,3: 0.8715355805243443
Average Recall for Random Forest model 1000,35: 0.8647435897435899
Average Accuracy for Random Forest model 1000,5: 0.8729962546816477
Average Recall for Random Forest model 1000,7: 0.8644871794871796
Average Accuracy for Random Forest model 1000,7: 0.902921348314607


In [9]:
print('Average Recall for Random Forest model 1500,3:', np.mean(rf_1500_3_recall))
print('Average Accuracy for Random Forest model 1500,3:', np.mean(rf_1500_3_accuracy))
print('Average Recall for Random Forest model 1500,35:', np.mean(rf_1500_5_recall))
print('Average Accuracy for Random Forest model 1500,5:', np.mean(rf_1500_5_accuracy))
print('Average Recall for Random Forest model 1500,7:', np.mean(rf_1500_7_recall))
print('Average Accuracy for Random Forest model 1500,7:', np.mean(rf_1500_7_accuracy))

Average Recall for Random Forest model 1500,3: 0.8646153846153847
Average Accuracy for Random Forest model 1500,3: 0.8715543071161046
Average Recall for Random Forest model 1500,35: 0.8647435897435899
Average Accuracy for Random Forest model 1500,5: 0.8728651685393255
Average Recall for Random Forest model 1500,7: 0.8643589743589745
Average Accuracy for Random Forest model 1500,7: 0.903239700374532


In [ ]:
print('Average Recall for Random Forest model 2000,3:', np.mean(rf_2000_3_recall))
print('Average Accuracy for Random Forest model 2000,3:', np.mean(rf_2000_3_accuracy))
print('Average Recall for Random Forest model 2000,35:', np.mean(rf_2000_5_recall))
print('Average Accuracy for Random Forest model 2000,5:', np.mean(rf_2000_5_accuracy))
print('Average Recall for Random Forest model 2000,7:', np.mean(rf_2000_7_recall))
print('Average Accuracy for Random Forest model 2000,7:', np.mean(rf_2000_7_accuracy))

In [5]:
ada_100_3_001_recall= list()
ada_100_3_001_accuracy= list()
ada_100_3_01_recall= list()
ada_100_3_01_accuracy= list()
ada_100_3_1_recall= list()
ada_100_3_1_accuracy= list()


ada_100_5_001_recall= list()
ada_100_5_001_accuracy= list()
ada_100_5_01_recall= list()
ada_100_5_01_accuracy= list()
ada_100_5_1_recall= list()
ada_100_5_1_accuracy= list()

ada_100_7_001_recall= list()
ada_100_7_001_accuracy= list()
ada_100_7_01_recall= list()
ada_100_7_01_accuracy= list()
ada_100_7_1_recall= list()
ada_100_7_1_accuracy= list()

ada_500_3_001_recall= list()
ada_500_3_001_accuracy= list()
ada_500_3_01_recall= list()
ada_500_3_01_accuracy= list()
ada_500_3_1_recall= list()
ada_500_3_1_accuracy= list()

ada_500_5_001_recall= list()
ada_500_5_001_accuracy= list()
ada_500_5_01_recall= list()
ada_500_5_01_accuracy= list()
ada_500_5_1_recall= list()
ada_500_5_1_accuracy= list()

ada_500_7_001_recall= list()
ada_500_7_001_accuracy= list()
ada_500_7_01_recall= list()
ada_500_7_01_accuracy= list()
ada_500_7_1_recall= list()
ada_500_7_1_accuracy= list()

ada_1000_3_001_accuracy= list()
ada_1000_3_001_recall= list()
ada_1000_3_01_accuracy= list()
ada_1000_3_01_recall= list()
ada_1000_3_1_accuracy= list()
ada_1000_3_1_recall= list()

ada_1000_5_001_recall= list()
ada_1000_5_001_accuracy= list()
ada_1000_5_01_recall= list()
ada_1000_5_01_accuracy= list()
ada_1000_5_1_recall= list()
ada_1000_5_1_accuracy= list()

ada_1000_7_001_recall= list()
ada_1000_7_001_accuracy= list()
ada_1000_7_01_recall= list()
ada_1000_7_01_accuracy= list()
ada_1000_7_1_recall= list()
ada_1000_7_1_accuracy= list()


ada_1500_3_001_recall= list()
ada_1500_3_001_accuracy= list()
ada_1500_3_01_recall= list()
ada_1500_3_01_accuracy= list()
ada_1500_3_1_recall= list()
ada_1500_3_1_accuracy= list()

ada_1500_5_001_recall= list()
ada_1500_5_001_accuracy= list()
ada_1500_5_01_recall= list()
ada_1500_5_01_accuracy= list()
ada_1500_5_1_recall= list()
ada_1500_5_1_accuracy= list()

ada_1500_7_001_recall= list()
ada_1500_7_001_accuracy= list()
ada_1500_7_01_recall= list()
ada_1500_7_01_accuracy= list()
ada_1500_7_1_recall= list()
ada_1500_7_1_accuracy= list()


ada_2000_3_001_recall= list()
ada_2000_3_001_accuracy= list()
ada_2000_3_01_recall= list()
ada_2000_3_01_accuracy= list()
ada_2000_3_1_recall= list()
ada_2000_3_1_accuracy= list()

ada_2000_5_001_recall= list()
ada_2000_5_001_accuracy= list()
ada_2000_5_01_recall= list()
ada_2000_5_01_accuracy= list()
ada_2000_5_1_recall= list()
ada_2000_5_1_accuracy= list()

ada_2000_7_001_recall= list()
ada_2000_7_001_accuracy= list()
ada_2000_7_01_recall= list()
ada_2000_7_01_accuracy= list()
ada_2000_7_1_recall= list()
ada_2000_7_1_accuracy= list()

In [ ]:
##### Building Ada models

for i in range(0,100):
    #Splitting the Data
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, stratify= y)

#### 100,3    
    #Building the model
    ada_md1= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 100, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred1= ada_md1.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels1= np.where(ada_pred1 < .1, 0, 1)
    ada_100_3_001_recall.append(recall_score(y_test, ada_labels1))
    ada_100_3_001_accuracy.append(accuracy_score(y_test, ada_labels1))
    
    #Building the model
    ada_md2= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 100, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred2= ada_md2.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels2= np.where(ada_pred2 < .1, 0, 1)
    ada_100_3_01_recall.append(recall_score(y_test, ada_labels2))
    ada_100_3_01_accuracy.append(accuracy_score(y_test, ada_labels2))
    
    #Building the model
    ada_md3= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 100, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred3= ada_md3.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels3= np.where(ada_pred3 < .1, 0, 1)
    ada_100_3_1_recall.append(recall_score(y_test, ada_labels3))
    ada_100_3_1_accuracy.append(accuracy_score(y_test, ada_labels3))
    
    
### 100,5
    #Building the model
    ada_md4= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 100, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred4= ada_md4.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels4= np.where(ada_pred1 < .1, 0, 1)
    ada_100_5_001_recall.append(recall_score(y_test, ada_labels4))
    ada_100_5_001_accuracy.append(accuracy_score(y_test, ada_labels4))
    
    #Building the model
    ada_md5= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 100, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred5= ada_md5.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels5= np.where(ada_pred5 < .1, 0, 1)
    ada_100_5_01_recall.append(recall_score(y_test, ada_labels5))
    ada_100_5_01_accuracy.append(accuracy_score(y_test, ada_labels5))
    
    #Building the model
    ada_md6= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 100, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred6= ada_md6.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels6= np.where(ada_pred6 < .1, 0, 1)
    ada_100_5_1_recall.append(recall_score(y_test, ada_labels6))
    ada_100_5_1_accuracy.append(accuracy_score(y_test, ada_labels6))
    
    
    
### 100,7
    #Building the model
    ada_md7= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 100, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred7= ada_md7.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels7= np.where(ada_pred7 < .1, 0, 1)
    ada_100_7_001_recall.append(recall_score(y_test, ada_labels7))
    ada_100_7_001_accuracy.append(accuracy_score(y_test, ada_labels7))
    
    #Building the model
    ada_md8= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 100, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred8= ada_md8.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels8= np.where(ada_pred8 < .1, 0, 1)
    ada_100_7_01_recall.append(recall_score(y_test, ada_labels8))
    ada_100_7_01_accuracy.append(accuracy_score(y_test, ada_labels8))
    
    #Building the model
    ada_md9= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 100, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred9= ada_md9.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels9= np.where(ada_pred9 < .1, 0, 1)
    ada_100_7_1_recall.append(recall_score(y_test, ada_labels9))
    ada_100_7_1_accuracy.append(accuracy_score(y_test, ada_labels9))
    
    
    

    #### 500,3    
    #Building the model
    ada_md10= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 500, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred10= ada_md10.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels10= np.where(ada_pred10 < .1, 0, 1)
    ada_500_3_001_recall.append(recall_score(y_test, ada_labels10))
    ada_500_3_001_accuracy.append(accuracy_score(y_test, ada_labels10))
    
    #Building the model
    ada_md11= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred11= ada_md11.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels11= np.where(ada_pred11 < .1, 0, 1)
    ada_500_3_01_recall.append(recall_score(y_test, ada_labels11))
    ada_500_3_01_accuracy.append(accuracy_score(y_test, ada_labels11))
    
    #Building the model
    ada_md12= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators=500, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred12= ada_md12.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels12= np.where(ada_pred12 < .1, 0, 1)
    ada_500_3_1_recall.append(recall_score(y_test, ada_labels12))
    ada_500_3_1_accuracy.append(accuracy_score(y_test, ada_labels12))
    
    
### 500,5
    #Building the model
    ada_md13= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 500, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred13= ada_md13.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels13= np.where(ada_pred13 < .1, 0, 1)
    ada_500_5_001_recall.append(recall_score(y_test, ada_labels13))
    ada_500_5_001_accuracy.append(accuracy_score(y_test, ada_labels13))
    
    #Building the model
    ada_md14= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred14= ada_md14.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels14= np.where(ada_pred14 < .1, 0, 1)
    ada_500_5_01_recall.append(recall_score(y_test, ada_labels14))
    ada_500_5_01_accuracy.append(accuracy_score(y_test, ada_labels14))
    
    #Building the model
    ada_md15= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 500, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred15= ada_md15.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels15= np.where(ada_pred6 < .1, 0, 1)
    ada_500_5_1_recall.append(recall_score(y_test, ada_labels15))
    ada_500_5_1_accuracy.append(accuracy_score(y_test, ada_labels15))
    
    
    
### 500,7
    #Building the model
    ada_md16= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 500, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred16= ada_md7.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels16= np.where(ada_pred16 < .1, 0, 1)
    ada_500_7_001_recall.append(recall_score(y_test, ada_labels16))
    ada_500_7_001_accuracy.append(accuracy_score(y_test, ada_labels16))
    
    #Building the model
    ada_md17= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred17= ada_md17.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels17= np.where(ada_pred17 < .1, 0, 1)
    ada_500_7_01_recall.append(recall_score(y_test, ada_labels17))
    ada_500_7_01_accuracy.append(accuracy_score(y_test, ada_labels17))
    
    #Building the model
    ada_md45= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 500, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred45= ada_md45.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels45= np.where(ada_pred45 < .1, 0, 1)
    ada_500_7_1_recall.append(recall_score(y_test, ada_labels45))
    ada_500_7_1_accuracy.append(accuracy_score(y_test, ada_labels45))  
    
    
    
    
    
#### 1000,3    
    #Building the model
    ada_md18= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 1000, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred18= ada_md18.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels18= np.where(ada_pred18 < .1, 0, 1)
    ada_1000_3_001_recall.append(recall_score(y_test, ada_labels18))
    ada_1000_3_001_accuracy.append(accuracy_score(y_test, ada_labels18))
    
    #Building the model
    ada_md19= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 1000, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred19= ada_md19.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels19= np.where(ada_pred19 < .1, 0, 1)
    ada_1000_3_01_recall.append(recall_score(y_test, ada_labels19))
    ada_1000_3_01_accuracy.append(accuracy_score(y_test, ada_labels19))
    
    #Building the model
    ada_md20= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 1000, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred20= ada_md20.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels20= np.where(ada_pred20 < .1, 0, 1)
    ada_1000_3_1_recall.append(recall_score(y_test, ada_labels20))
    ada_1000_3_1_accuracy.append(accuracy_score(y_test, ada_labels20))
    
    
### 1000,5
    #Building the model
    ada_md21= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 1000, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred21= ada_md21.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels21= np.where(ada_pred21 < .1, 0, 1)
    ada_1000_5_001_recall.append(recall_score(y_test, ada_labels21))
    ada_1000_5_001_accuracy.append(accuracy_score(y_test, ada_labels21))
    
    #Building the model
    ada_md22= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 1000, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred22= ada_md22.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels22= np.where(ada_pred22 < .1, 0, 1)
    ada_500_5_01_recall.append(recall_score(y_test, ada_labels22))
    ada_500_5_01_accuracy.append(accuracy_score(y_test, ada_labels22))
    
    #Building the model
    ada_md23= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 1000, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred23= ada_md23.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels23= np.where(ada_pred23 < .1, 0, 1)
    ada_500_5_1_recall.append(recall_score(y_test, ada_labels23))
    ada_500_5_1_accuracy.append(accuracy_score(y_test, ada_labels23))
    
    
    
### 1000,7
    #Building the model
    ada_md24= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 1000, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred24= ada_md24.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels24= np.where(ada_pred24 < .1, 0, 1)
    ada_1000_7_001_recall.append(recall_score(y_test, ada_labels24))
    ada_1000_7_001_accuracy.append(accuracy_score(y_test, ada_labels24))
    
    #Building the model
    ada_md25= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 1000, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred25= ada_md25.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels25= np.where(ada_pred25 < .1, 0, 1)
    ada_1000_7_01_recall.append(recall_score(y_test, ada_labels25))
    ada_1000_7_01_accuracy.append(accuracy_score(y_test, ada_labels25))
    
    #Building the model
    ada_md26= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 1000, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred26= ada_md26.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels26= np.where(ada_pred26 < .1, 0, 1)
    ada_1000_7_1_recall.append(recall_score(y_test, ada_labels26))
    ada_1000_7_1_accuracy.append(accuracy_score(y_test, ada_labels26))    
    
    
    
    
    
    
#### 1500,3    
    #Building the model
    ada_md27= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 1500, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred27= ada_md27.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels27= np.where(ada_pred27 < .1, 0, 1)
    ada_1500_3_001_recall.append(recall_score(y_test, ada_labels27))
    ada_1500_3_001_accuracy.append(accuracy_score(y_test, ada_labels27))
    
    #Building the model
    ada_md28= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 1500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred28= ada_md28.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels28= np.where(ada_pred28 < .1, 0, 1)
    ada_1500_3_01_recall.append(recall_score(y_test, ada_labels28))
    ada_1500_3_01_accuracy.append(accuracy_score(y_test, ada_labels28))
    
    #Building the model
    ada_md29= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 1500, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred29= ada_md29.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels29= np.where(ada_pred29 < .1, 0, 1)
    ada_1500_3_1_recall.append(recall_score(y_test, ada_labels29))
    ada_1500_3_1_accuracy.append(accuracy_score(y_test, ada_labels29))
    
    
### 1500,5
    #Building the model
    ada_md30= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 1500, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred30= ada_md30.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels30= np.where(ada_pred30 < .1, 0, 1)
    ada_1500_5_001_recall.append(recall_score(y_test, ada_labels30))
    ada_1500_5_001_accuracy.append(accuracy_score(y_test, ada_labels30))
    
    #Building the model
    ada_md31= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 1500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred31= ada_md31.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels31= np.where(ada_pred31 < .1, 0, 1)
    ada_1500_5_01_recall.append(recall_score(y_test, ada_labels31))
    ada_1500_5_01_accuracy.append(accuracy_score(y_test, ada_labels31))
    
    #Building the model
    ada_md32= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 1500, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred32= ada_md32.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels32= np.where(ada_pred32 < .1, 0, 1)
    ada_1500_5_1_recall.append(recall_score(y_test, ada_labels32))
    ada_1500_5_1_accuracy.append(accuracy_score(y_test, ada_labels32))
    
    
    
### 1500,7
    #Building the model
    ada_md33= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 1500, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred33= ada_md33.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels33= np.where(ada_pred33 < .1, 0, 1)
    ada_1500_7_001_recall.append(recall_score(y_test, ada_labels33))
    ada_1500_7_001_accuracy.append(accuracy_score(y_test, ada_labels33))
    
    #Building the model
    ada_md34= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 1500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred34= ada_md34.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels34= np.where(ada_pred34 < .1, 0, 1)
    ada_1500_7_01_recall.append(recall_score(y_test, ada_labels34))
    ada_1500_7_01_accuracy.append(accuracy_score(y_test, ada_labels34))
    
    #Building the model
    ada_md35= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 1500, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred35= ada_md35.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels35= np.where(ada_pred35 < .1, 0, 1)
    ada_1500_7_1_recall.append(recall_score(y_test, ada_labels35))
    ada_1500_7_1_accuracy.append(accuracy_score(y_test, ada_labels35))   
    
    
#### 2000,3  
    #Building the model
    ada_md36= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 2000, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred36= ada_md36.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels36= np.where(ada_pred36 < .1, 0, 1)
    ada_1500_3_001_recall.append(recall_score(y_test, ada_labels36))
    ada_1500_3_001_accuracy.append(accuracy_score(y_test, ada_labels36))
    
    #Building the model
    ada_md37= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 2000, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred37= ada_md37.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels37= np.where(ada_pred37 < .1, 0, 1)
    ada_2000_3_01_recall.append(recall_score(y_test, ada_labels37))
    ada_2000_3_01_accuracy.append(accuracy_score(y_test, ada_labels37))
    
    #Building the model
    ada_md38= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 2000, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred38= ada_md38.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels38= np.where(ada_pred38 < .1, 0, 1)
    ada_2000_3_1_recall.append(recall_score(y_test, ada_labels38))
    ada_2000_3_1_accuracy.append(accuracy_score(y_test, ada_labels38))
    
    
### 2000,5
    #Building the model
    ada_md39= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 2000, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred39= ada_md39.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels39= np.where(ada_pred39 < .1, 0, 1)
    ada_2000_5_001_recall.append(recall_score(y_test, ada_labels39))
    ada_2000_5_001_accuracy.append(accuracy_score(y_test, ada_labels39))
    
    #Building the model
    ada_md40= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 2000, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred40= ada_md40.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels40= np.where(ada_pred40 < .1, 0, 1)
    ada_2000_5_01_recall.append(recall_score(y_test, ada_labels40))
    ada_2000_5_01_accuracy.append(accuracy_score(y_test, ada_labels40))
    
    #Building the model
    ada_md41= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 5), n_estimators= 2000, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred41= ada_md41.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels41= np.where(ada_pred41 < .1, 0, 1)
    ada_2000_5_1_recall.append(recall_score(y_test, ada_labels41))
    ada_2000_5_1_accuracy.append(accuracy_score(y_test, ada_labels41))
    
    
    
### 2000,7
    #Building the model
    ada_md42= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 2000, learning_rate=.001).fit(x_train, y_train)
    #Predicting on the model
    ada_pred42= ada_md42.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels42= np.where(ada_pred42 < .1, 0, 1)
    ada_2000_7_001_recall.append(recall_score(y_test, ada_labels42))
    ada_2000_7_001_accuracy.append(accuracy_score(y_test, ada_labels42))
    
    #Building the model
    ada_md43= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 2000, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred43= ada_md43.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels43= np.where(ada_pred34 < .1, 0, 1)
    ada_2000_7_01_recall.append(recall_score(y_test, ada_labels43))
    ada_2000_7_01_accuracy.append(accuracy_score(y_test, ada_labels43))
    
    #Building the model
    ada_md44= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 7), n_estimators= 2000, learning_rate=.1).fit(x_train, y_train)
    #Predicting on the model
    ada_pred44= ada_md44.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels44= np.where(ada_pred44 < .1, 0, 1)
    ada_2000_7_1_recall.append(recall_score(y_test, ada_labels44))
    ada_2000_7_1_accuracy.append(accuracy_score(y_test, ada_labels44))   

In [ ]:
print('Average Recall for Ada model 2000,7,.1:', np.mean(ada_2000_7_1_recall))
print('Average Accuracy for Ada model 2000, 7,.1:', np.mean(ada_2000_7_1_accuracy))
